## AuxTel test LTS-337-003 (Nasmyth rotator)

In this notebook, I slew the Nasmyth2 rotator through it's range of travel, and evaluate range, max slew speed, and accuracy of position.  Desired specs:

| Description | Value       | Unit          |   Name     |
| :---        |    :----:   |       :----:  |       ---: |
|The rotator shall have a minimum range of rotation of:    | ±120       | Degrees   |Aux_Tel_Field_Rotation_Range|
|The rotator shall be able to achieve or surpass this velocity during slews of the telescope.  |3.5       | Degrees/Second      |Aux_Tel_Inst_Rot_Max_Vel|
|The rotator shall have at maximum this absolute angle error.      | 0.01|Degrees|Aux_Tel_Inst_Rot_Abs_Error|

In [1]:
import sys, time, os, asyncio

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient


Bad key "text.kerning_factor" on line 4 in
/opt/lsst/software/stack/conda/miniconda3-py37_4.8.2/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
# Get EFD client and bring in Lupton's unpacking code
client = EfdClient('summit_efd')

def merge_packed_time_series(packed_dataframe, base_field, stride=1, 
                             ref_timestamp_col="cRIO_timestamp", internal_time_scale="tai"):
    """Select fields that are time samples and unpack them into a dataframe.
            Parameters
            ----------
            packedDF : `pandas.DataFrame`
                packed data frame containing the desired data
            base_field :  `str`
                Base field name that will be expanded to query all
                vector entries.
            stride : `int`, optional
                Only use every stride value when unpacking.  Must be a factor
                of the number of packed values.
                (1 by default)
            ref_timestamp_col : `str`, optional
                Name of the field name to use to assign timestamps to unpacked
                vector fields (default is 'cRIO_timestamp').
            internal_time_scale : `str`, optional
                Time scale to use when converting times to internal formats
                ('tai' by default). Equivalent to EfdClient.internal_scale
        Returns
            -------
            result : `pandas.DataFrame`
                A `pandas.DataFrame` containing the results of the query.
            """
    
    packed_fields = [k for k in packed_dataframe.keys() if k.startswith(base_field)]
    packed_fields = sorted(packed_fields, key=lambda k: int(k[len(base_field):]))  # sort by pack ID
    npack = len(packed_fields)
    if npack%stride != 0:
        raise RuntimeError(f"Stride must be a factor of the number of packed fields: {stride} v. {npack}")
    packed_len = len(packed_dataframe)
    n_used = npack//stride   # number of raw fields being used
    output = np.empty(n_used*packed_len)
    times = np.empty_like(output, dtype=packed_dataframe[ref_timestamp_col][0])
    
    if packed_len == 1:
        dt = 0
    else:
        dt = (packed_dataframe[ref_timestamp_col][1] - packed_dataframe[ref_timestamp_col][0])/npack
    for i in range(0, npack, stride):
        i0 = i//stride
        output[i0::n_used] = packed_dataframe[f"{base_field}{i}"]
        times[i0::n_used] = packed_dataframe[ref_timestamp_col] + i*dt
     
    timestamps = Time(times, format='unix', scale=internal_time_scale).datetime64
    return pd.DataFrame({base_field:output, "times":times}, index=timestamps)

In [4]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.15 sec
Read 6 history items for RemoteEvent(ATAOS, 0, appliedSettingsMatchStart)
Read 19 history items for RemoteEvent(ATAOS, 0, atspectrographCorrectionCompleted)
Read 19 history items for RemoteEvent(ATAOS, 0, atspectrographCorrectionStarted)
Read 1 history items for RemoteEvent(ATAOS, 0, authList)
Read 7 history items for RemoteEvent(ATAOS, 0, correctionEnabled)
Read 16 history items for RemoteEvent(ATAOS, 0, correctionOffsets)
Read 100 history items for RemoteEvent(ATAOS, 0, detailedState)
Read 16 history items for RemoteEvent(ATAOS, 0, focusOffsetSummary)
Read 100 history items for RemoteEvent

[[None, None, None, None, None, None, None], [None, None, None, None]]

timeAndDate DDS read queue is filling: 95 of 100 elements


In [5]:
atcs.check.atdome = True

In [6]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.ENABLED)

RuntimeError: Error on cmd=cmd_enable, initial_state=1: msg='Command failed', ackcmd=(ackcmd private_seqNum=502243990, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: enable not allowed in state <State.STANDBY: 5>')

In [ ]:
await atcs.rem.atdome.cmd_start.set_start(settingsToApply="test", timeout=30)

In [10]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.ENABLED, settingsToApply="test")

[<State.DISABLED: 1>, <State.ENABLED: 2>]

In [11]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply="test")

RuntimeError: Error on cmd=cmd_standby, initial_state=2: msg='Command failed', ackcmd=(ackcmd private_seqNum=1831841208, ack=<SalRetCode.CMD_TIMEOUT: -304>, error=1, result='Timeout')

In [ ]:
# take event checking out of the slew commands to test telescope only
atcs.check.atdome = True
atcs.check.atdometrajectory = True

In [ ]:
atcs.shutdown?

In [ ]:
# Now shut down
await atcs.shutdown()

In [ ]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.ENABLED)

In [ ]:
tmp = atcs.rem.atdome.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.OFFLINE)

In [ ]:
await atcs.stop_tracking()

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.ataos, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY)

In [ ]:
# Check time synchronization
! zdump /etc/localtime ; date

In [ ]:
await atcs.enable()

In [ ]:
# Putting everything back in standby.
# Need to do it one at a time, since ATDome fails with check=False.
#await atcs.stop_tracking()
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.ataos, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)
await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.ENABLED)
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"Current time = {current_time}")
t1 = current_time + TimeDelta(-30, format='sec', scale='tai')
t2 = t1 + TimeDelta(60, format='sec')
df = await client.select_time_series('lsst.sal.ATMCS.logevent_summaryState', '*', t1, t2)
print(df['summaryState'])

In [ ]:
# Time problem seems fixed!

In [ ]:
# enable components if required
await atcs.enable()
#await latiss.enable()

In [ ]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [ ]:
# turn on ATAOS corrections just to make sure the mirror is under air
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=False)

In [ ]:
# Ensure we're using Nasmyth 2
await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [ ]:
# slew telescope to desired starting position
# rotator does not move for this test as it's part of a different
# requirement/verification
start_az=0
start_el=80
start_rot_pa=0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
# Here is where the tests are carried out.
max_rot = 10.0 # Maximum +/- 130 degrees
rot_step = 10.0 # 10 degree steps
n_steps = int(max_rot / rot_step)
maxes = []
errors = []
speeds = []
signs = [-1.0, 1.0] # Repeat tests in both directions

for sign in signs:
    await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)
    # Move to max_rot
    current_rot = start_rot_pa + sign * max_rot
    await atcs.point_azel(start_az, start_el, rot_tel=current_rot, wait_dome=False)
    current_time = Time(datetime.now().strftime("%Y-%m-%dT%H:%M:%-S"), scale='tai')
    # Back up 10 seconds and get angle data for 10 seconds before that
    t_end = current_time + TimeDelta(-10.0, format='sec', scale='tai')
    nsec = 10
    nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                              t_end - TimeDelta(nsec, format='sec'), t_end)
    angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
    angleList = angle.values.tolist()
    velocity = (angleList[-1][0] - angleList[0][0]) / (angleList[-1][1] - angleList[0][1])
    print(f"Measured velocity = {velocity}")
    speeds.append(abs(velocity))
    # Get current position to verify it is beyond the max
    await asyncio.sleep(3)
    current_time = Time(datetime.now().strftime("%Y-%m-%dT%H:%M:%-S"), scale='tai')
    t_end = current_time
    nsec = 2
    nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                              t_end - TimeDelta(nsec, format='sec'), t_end)
    angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
    angleList = angle.values.tolist()
    current_angle = angleList[-1][0]
    print(f"Current rotator angle = {current_angle}")
    maxes.append(abs(current_angle))
    # Now slew through a range of angles and evaluate the error
    for n in range(n_steps):
        current_rot = current_rot - sign * rot_step
        await atcs.point_azel(start_az, start_el, rot_tel=current_rot, wait_dome=False)
        # Get current position and compare to set point
        await asyncio.sleep(3)
        current_time = Time(datetime.now().strftime("%Y-%m-%dT%H:%M:%-S"), scale='tai')
        t_end = current_time
        nsec = 2
        nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                                  t_end - TimeDelta(nsec, format='sec'), t_end)
        angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
        angleList = angle.values.tolist()
        current_angle = angleList[-1][0]
        error = current_angle - current_rot
        print(f"Current rotator angle = {current_angle}. Set point = {current_rot}.  Error = {error}")
        errors.append(abs(error))


In [ ]:
current_time = Time(datetime.now().strftime("%Y-%m-%dT%H:%M:%-S"), scale='tai')
print(current_time)
t_end = current_time
nsec = 2
nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
angleList = angle.values.tolist()
current_angle = angleList[-1][0]
print(current_angle)

In [ ]:
start_az=0
start_el=80
start_rot_pa=0.0
max_rot = 30.0
sign = -1.0
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"Before first move (tai) = {current_time}")
print(f"Before first move (utc) = {Time(Time.now(), format='fits', scale='utc')}")
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"After first move = {current_time}")
print(f"After first move (utc) = {Time(Time.now(), format='fits', scale='utc')}")
# Move to max_rot
current_rot = start_rot_pa + sign * max_rot
await atcs.point_azel(start_az, start_el, rot_tel=current_rot, wait_dome=False)
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"After second move = {current_time}")
print(f"After second move (utc) = {Time(Time.now(), format='fits', scale='utc')}")

# Back up 30 seconds and get angle data for 5 seconds before that
t_end = current_time + TimeDelta(-30.0, format='sec', scale='tai')
print(f"t_end = {t_end}")
nsec = 5
nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
angleList = angle.values.tolist()
velocity = (angleList[-1][0] - angleList[0][0]) / (angleList[-1][1] - angleList[0][1])
print(f"Measured velocity = {velocity}")

current_time = Time(Time.now(), format='fits', scale='tai')
t_end = current_time
nsec = 2
nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
angleList = angle.values.tolist()
print(f"Current angle = {angleList[-1][0]}")

In [ ]:
current_time = Time(Time.now(), format='fits', scale='tai')
t_end = current_time
nsec = 2
nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)
angleList = angle.values.tolist()
print(f"Current angle = {angleList[-1][0]}")

In [ ]:
t_end = Time("2021-02-09T11:50:18.796", scale='tai')   # timestamp at end of desired data 
nsec = 90 # how many seconds of data to retrieve
nasmyth_angle = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
angle = merge_packed_time_series(nasmyth_angle, 'nasmyth2CalculatedAngle', stride=1)




In [ ]:
fig = angle['nasmyth2CalculatedAngle'].plot(legend=True)
plt.axvline(pd.to_datetime('2021-02-09T11:50:18.796'), color='r', linestyle='--', lw=2)
plt.text(pd.to_datetime('2021-02-09T11:50:18.796'), -15, "Move finished (tai)", color='r')
#plt.axvline(pd.to_datetime('2021-02-08T18:45:28.436'), color='g', linestyle='--', lw=2)
#plt.text(pd.to_datetime('2021-02-08T18:45:28.436'), -15, "Move finished (utc)", color='g')
plt.title("Move from 0.0 to -30.0 degrees")
#plt.savefig("/home/craiglagegit/DATA/Rotator_Test_08Feb21.pdf")

In [ ]:
nasmyth_velocity = await client.select_time_series("lsst.sal.ATMCS.measuredMotorVelocity", ['*'],
                                              t_end - TimeDelta(nsec, format='sec'), t_end)
velocity = merge_packed_time_series(nasmyth_velocity, 'nasmyth2MotorVelocity', stride=1)
velocity['nasmyth2MotorVelocity'].plot(legend=True)
velArray = np.array(angle.values.tolist())[:,0]
sorted_index_array = np.argsort(arr) 
  
# sorted array 
sorted_array = arr[sorted_index_array] 
  
print("Sorted array:", sorted_array) 
  
# we want 1 largest value 
n = 1
  
# we are using negative 
# indexing concept 
  
# take n largest value 
rslt = sorted_array[-n : ] 


In [ ]:
velArray = np.array(velocity.values.tolist())[:,0]
sortedVelArray = velArray[np.argsort(velArray)]
print(np.median(sortedVelArray[0:100]))

In [ ]:
t1 = Time('2021-02-08T18:45:30', scale='tai')
t2 = t1 + TimeDelta(120, format='sec', scale='tai')
df = await client.select_time_series('lsst.sal.ATMCS.logevent_nasmyth2RotatorInPosition', '*', t1, t2)
print(df)

In [ ]:
df = await client.select_time_series('lsst.sal.ATMCS.logevent_nasmyth2RotatorInPosition', '*', t1, t2)
print(df)

In [ ]:
df = await client.select_time_series('lsst.sal.ATMCS.logevent_nasmyth2RotatorInPosition.private_sndStamp', '*', t1, t2)
print(df)

In [ ]:
angleList = angle.values.tolist()
print(f"Current angle = {angleList[-1][0]}")

In [ ]:
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"Current time = {current_time}")


# Back up 30 seconds and get angle data for 5 seconds before that
t_end = current_time + TimeDelta(-30.0, format='sec', scale='tai')
print(f"t_end = {t_end}")

In [ ]:
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"Current time = {current_time}")
current_time = Time(Time.now(), format='fits', scale='utc')
print(f"Current time = {current_time}")


In [ ]:
len(angleList)
angleList

In [ ]:
print(Time(datetime.now(), scale='utc'))
print(Time(datetime.now(), scale='tai'))

In [ ]:
print(Time(Time.now(), format='fits', scale='utc'))
print(Time(Time.now(), format='fits', scale='tai'))

In [ ]:
# Now check to see if the specs are met:
Aux_Tel_Field_Rotation_Range = 120.0
if min(maxes) > Aux_Tel_Field_Rotation_Range:
    print(f"Aux_Tel_Field_Rotation_Range passed.  Spec = {Aux_Tel_Field_Rotation_Range}. Measured = {min(maxes)} ")
else:
    print(f"Aux_Tel_Field_Rotation_Range failed!  Spec = {Aux_Tel_Field_Rotation_Range}. Measured = {min(maxes)} ")

Aux_Tel_Inst_Rot_Max_Vel = 3.5
if min(speeds) > Aux_Tel_Inst_Rot_Max_Vel:
    print(f"Aux_Tel_Inst_Rot_Max_Vel passed.  Spec = {Aux_Tel_Inst_Rot_Max_Vel}. Measured = {min(speeds)} ")
else:
    print(f"Aux_Tel_Inst_Rot_Max_Vel failed!  Spec = {Aux_Tel_Inst_Rot_Max_Vel}. Measured = {min(speeds)} ")

Aux_Tel_Inst_Rot_Abs_Error = 0.01
if max(errors) < Aux_Tel_Inst_Rot_Abs_Error:
    print(f"Aux_Tel_Inst_Rot_Abs_Error passed.  Spec = {Aux_Tel_Inst_Rot_Abs_Error}. Worst case error = {max(errors)} ")
else:
    print(f"Aux_Tel_Inst_Rot_Abs_Error failed!  Spec = {Aux_Tel_Inst_Rot_Abs_Error}. Worst case error = {max(errors)} ")


In [ ]:
# take event checking out of the slew commands to test telescope only
atcs.check.atdome = True
atcs.check.atdometrajectory = True

In [ ]:
# Now shut down
await atcs.shutdown()

In [ ]:
atcs.check.atdome = False
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.ENABLED)
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"Current time = {current_time}")

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"Current time = {current_time}")
t1 = current_time + TimeDelta(-30, format='sec', scale='tai')
t2 = t1 + TimeDelta(60, format='sec')
df = await client.select_time_series('lsst.sal.ATMCS.logevent_summaryState', '*', t1, t2)
print(df['summaryState'])

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.ENABLED)
current_time = Time(Time.now(), format='fits', scale='utc')
print(f"Current time = {current_time}")
t1 = current_time + TimeDelta(-30, format='sec')
t2 = t1 + TimeDelta(60, format='sec')
df = await client.select_time_series('lsst.sal.ATMCS.logevent_summaryState', '*', t1, t2)
print(df['summaryState'])

In [ ]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.ENABLED)
current_time = Time(Time.now(), format='fits', scale='tai')
print(f"Current time = {current_time}")
t1 = current_time + TimeDelta(-30, format='sec', scale='tai')
t2 = t1 + TimeDelta(60, format='sec', scale='tai')
df = await client.select_time_series('lsst.sal.ATDomeTrajectory.logevent_summaryState', '*', t1, t2)
print(df['summaryState'])